<a href="https://colab.research.google.com/github/kisa-misa/i-love-dogs/blob/main/_question_answering_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question answering (PyTorch)

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the following line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!apt install git-lfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

You will need to setup git, adapt your email and name in the following cell.

In [ ]:
!git config --global user.email "Zhbanckova.a@yandex.ru"
!git config --global user.name "Anya"

You will also need to be logged in to the Hugging Face Hub. Execute the following and enter your credentials.

In [ ]:
from datasets import load_dataset

data = load_dataset('json', data_files='/content/train.json', split="train")
data = data.train_test_split(test_size=0.2)
train_data = data['train']
val_data = data['test']
test_data = load_dataset('json', data_files='/content/test.json')['train']

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-693247354f9e2a79/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-acd62e3de52ac41a/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
questions = ["Какой фрагмент текста соответствует размеру {}?".format(label) for label in train_data['label']]
train_data = train_data.remove_columns("label").add_column("question", questions)
train_data[1]
val_questions = ["Какой фрагмент текста соответствует размеру {}?".format(label) for label in val_data['label']]
val_data = val_data.remove_columns("label").add_column("question", val_questions)

Flattening the indices:   0%|          | 0/1439 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/360 [00:00<?, ? examples/s]

In [ ]:
print("Context: ", train_data[1]["text"])
print("Question: ", train_data[1]["question"])
print("Answer: ", train_data[1]['extracted_part'])

Context:  Извещение о проведении электронного аукциона для закупки №0308300024722000155 Основание Решение Заказчика (организации, осуществляющей определение поставщика (подрядчика, исполнителя) для заказчика) от 05.09.2022 Дополнительная информация Информация отсутствует Краткое описание изменения Корректировка Приложения ние допуска и условия допуска Постановление Правительства РФ от 30.11.2015 N 1289 "Об ограничениях и условиях допуска происходящих из иностранных государств лекарственных препаратов, включенных в перечень жизненно необходимых и важнейших лекарственных препаратов, для целей осуществления закупок для обеспечения государственных и муниципальных нужд" Установлены ограничения допуска происходящих из иностранных государств лекарственных препаратов, включенных в перечень жизненно необходимых и важнейших лекарственных препаратов (постановление Правительства Российской Федерации от 30 ноября 2015 г. № 1289 «Об ограничениях и условиях допуска происходящих из иностранных государ

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "DeepPavlov/distilrubert-tiny-cased-conversational"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
context = train_data[0]["text"]
question = train_data[0]["question"]

inputs = tokenizer(question, context)
tokenizer.decode(inputs["input_ids"])

'[CLS] Какой фрагмент текста соответствует размеру обеспечение гарантийных обязательств? [SEP] ПРОЕКТ МУНИЦИПАЛЬНОГО КОНТРАКТА МУНИЦИПАЛЬНЫЙ КОНТРАКТ № _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ Поставка, установка, наладка электронной техники необходимой для оснащения объекта : « Общеобразовательная школа на 1100 мест в районе Кальное г. Рязани ». лучаях, которые предусмотрены пунктами 7. 1, 7. 5, 7. 6 и 7. 7 Контракта. 7. 9. Уменьшение в соответствии с пунктами 7. 1 и 7. 5 Контракта размера обеспечения исполнения Контракта, предоставленного в виде независимой гарантии, осуществляется Заказчиком путем отказа от части своих прав по этой гарантии. При этом датой такого отказа признается дата включения предусмотренной пунктом 7. 6 Контракта информации в реестр контрактов. 7. 10. В случае предоставления нового обеспечения исполнения Контракта в соответствии с пунктами 7. 5 и 7. 8 Контракта возврат независимой гарантии Заказчиком гаранту, предоставившему указанную независимую гарантию, не о

In [ ]:
max_length = 384
stride = 128


def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["text"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["extracted_part"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_end"][0]
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
train_dataset = train_data.map(
    preprocess_training_examples,
    batched=True,
    remove_columns=train_data.column_names,
)
len(train_data), len(train_dataset)

Map:   0%|          | 0/1439 [00:00<?, ? examples/s]

(1439, 2823)

In [ ]:
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        examples['question'],
        examples["text"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [ ]:
validation_dataset = val_data.map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=val_data.column_names,
)
len(val_data), len(validation_dataset)

Map:   0%|          | 0/360 [00:00<?, ? examples/s]

(360, 707)

In [ ]:
from tqdm.auto import tqdm
import collections
n_best = 20
max_answer_length = 100
import numpy as np
import evaluate

metric = evaluate.load('squad')

def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["text"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": str(example_id), "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": str(ex["id"]), "answers": {'text': ex["extracted_part"]['text'], 'answer_start': ex["extracted_part"]['answer_start']}} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at DeepPavlov/distilrubert-tiny-cased-conversational were not used when initializing DistilBertForQuestionAnswering: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at DeepPavlov/distilrubert-tiny-cased-conversational and are newly initialized: ['qa_outputs.weight', 'q

In [ ]:
# from huggingface_hub import notebook_login

# notebook_login()

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    "DeepPavlov/distilrubert-tiny-cased-conversational",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    #push_to_hub=True,
    #fp16=True,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
)
trainer.train()

Cloning https://huggingface.co/kisa-misa/distilrubert-tiny-cased-conversational into local empty directory.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,1.322300
1000,0.667500


TrainOutput(global_step=1059, training_loss=0.9731615983280402, metrics={'train_runtime': 8423.9108, 'train_samples_per_second': 1.004, 'train_steps_per_second': 0.126, 'total_flos': 276371251845120.0, 'train_loss': 0.9731615983280402, 'epoch': 3.0})

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("kisa-misa/distilrubert-tiny-cased-conversational")

model = AutoModelForQuestionAnswering.from_pretrained("kisa-misa/distilrubert-tiny-cased-conversational")


In [ ]:
predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions
compute_metrics(start_logits, end_logits, validation_dataset, val_data)

NameError: ignored

In [ ]:
#trainer.push_to_hub(commit_message="Training complete")

In [ ]:
#вставьте свой model_checkpoint
model = AutoModelForQuestionAnswering.from_pretrained('/content/DeepPavlov/distilrubert-tiny-cased-conversational/checkpoint-1059')

In [ ]:
from transformers import pipeline
#вставьте свой model_checkpoint
model_checkpoint = '/content/DeepPavlov/distilrubert-tiny-cased-conversational/checkpoint-1059' 
question_answerer = pipeline("question-answering", model=model_checkpoint)

context = """
Извещение о проведении электронного аукциона для закупки №0318100007022000091 Общая информация Номер извещения 0318100007022000091 Наименование объекта закупки Поставка расходных материалов Способ определения поставщика (подрядчика, исполнителя) Электронный аукцион Наименование электронной площадки в зованием электронной площадки способ обеспечения заявки путем указания реквизитов специального счета или указания номера реестровой записи из реестра независимых гарантий, размещенного в единой информационной системе в сфере закупок. Получатель: УФК по Краснодарскому краю (Краснодарский университет МВД России (КрУ МВД России) л/с 05181854470), Банк получателя: ЮЖНОЕ ГУ БАНКА РОССИИ//УФК по Краснодарскому краю г. Краснодар, БИК 010349101, ИНН 2311092752, КПП 231101001, ОКПО 08726067, ОКОНХ 92110, ОКТМО 03701000001, ЕКС (единый казначейский счет) 40102810945370000010, казначейский счет 03212643000000011800, В ПЛАТЕЖНОМ ПОРУЧЕНИИ В ПОЛЕ 22 ОБЯЗАТЕЛЬНО УКАЗЫВАЕТСЯ КОД 0002 Реквизиты счета в соответствии с п.16 ч. 1 ст. 42 Закона № 44-ФЗ \"Номер расчётного счёта\" 00000000000000000000 \"Номер лицевого счёта\" См. прилагаемые документы \"БИК\" 000000000 \"Наименование кредитной организации\" \"Номер корреспондентского счета\" Обеспечение исполнения контракта Требуется обеспечение исполнения контракта Размер обеспечения исполнения контракта 10.00% Порядок обеспечения исполнения контракта, требования к обеспечению Контракт заключается после предоставления участником закупки, с которым заключается контракт, обеспечения исполнения контракта в соответствии с требованиями статьи 96 Федерального закона от 05.04.2013 № 44-ФЗ «О контрактной системе в сфере закупок товаров, работ, услуг для обеспечения государственных и муниципальных нужд». Исполнение контракта обеспечивается предоставлением независимой гарантии, соответствующей требованиям статьи 45 Федерального закона от 05.04.2013 № 44-ФЗ «О контрактной системе в сфере закупок товаров, работ, услуг для обеспечения государственных и муниципальных нужд», или внесением денежных средств на указанный заказчиком счет с указанием назначения платежа «Обеспечение исполнения контракта, № закупки или ИКЗ». Способ обеспечения исполнения контракта определяется участником закупки, с которым заключается контракт, самостоятельно. Получатель: УФК по Краснодарскому краю (Краснодарский университет МВД  Обеспечение гарантийных обязательств Обеспечение гарантийных обязательств не требуется Дополнительная информация Информация отсутствует Информация о банковском и (или) казначейском сопровождении контракта Банковское или казначейское сопровождение контракта не требуется Перечень прикрепленных документов
"""
question = "обеспечение исполнения контракта"
question_answerer(question=question, context=context)

{'score': 0.9922049641609192,
 'start': 1303,
 'end': 1349,
 'answer': 'Размер обеспечения исполнения контракта 10.00%'}

In [ ]:
test_data = load_dataset('json', data_files='/content/test.json')['train']
def test_question_answerer(examples):
    outputs = []

    for ex in examples:
      label = ex['label']
      question = "Какой фрагмент текста соответствует размеру {}?".format(label)
      context = ex['text']
      answer = question_answerer(question=question, context=context)
      output = {"id": ex['id'], 'text': context, 'label': label, "extracted_part": {"text": [answer['answer']], 'answer_start': [answer['start']], 'answer_end': [answer['end']]}}
      outputs.append(output)

    return outputs


test_answers = test_question_answerer(test_data)

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:

import pandas as pd
from datasets import load_dataset
import torch
import json
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
with open('predictions.json', 'w') as f:
    json.dump(test_answers, f, ensure_ascii=False)
